# Preparation of GWR - Data

## Libraries and Settings

In [1]:
# Libraries
import os
import json
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import date

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/workspaces/spatial_data_analysis/05_Python_GWR_Data


## Download GWR data

Source: https://public.madd.bfs.admin.ch

In [9]:
# Download latest file for the Canton of Zurich
url = "https://public.madd.bfs.admin.ch/buildings_ow.geojson"
response = requests.get(url)

# Ensure response is valid
if response.status_code == 200:
    # Open file in write mode and write the response content
    with open('buildings_zh.geojson', 'wb') as file:
        file.write(response.content)
else:
    print(f"Failed to download file, status code: {response.status_code}")

## Import latest GWR Data

In [10]:
# Load json file
with open('buildings_zh.geojson') as f:
    data = json.load(f)

# Flatten nested json data
df_orig = pd.json_normalize(data, record_path=['features'])
df_orig

# Remove prefix
df_orig.columns = df_orig.columns.str.replace('properties.', '')
df_orig.columns = df_orig.columns.str.replace('geometry.', '')

# Create copy
df = df_orig.copy()
df.head()

,type,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,type,coordinates
0,Feature,101235305,1007,1020,1110,1401,Alpnach,OW,Point,"[2662391, 1198346]"
1,Feature,101235307,1004,1020,1110,1401,Alpnach,OW,Point,"[2662273.495, 1198664.438]"
2,Feature,101235308,1004,1020,1122,1401,Alpnach,OW,Point,"[2663069.775, 1200010.861]"
3,Feature,101235310,1004,1020,1110,1401,Alpnach,OW,Point,"[2662170.316, 1198226.835]"
4,Feature,101235311,1004,1020,1110,1401,Alpnach,OW,Point,"[2662135.736, 1198271.709]"


## Separate Swiss LV95 coordinates

In [11]:
# Separate coordinates
df['x_coords'] = pd.DataFrame(df['coordinates'].tolist(), columns=['x_coord', 'y_coord'])['x_coord']
df['y_coords'] = pd.DataFrame(df['coordinates'].tolist(), columns=['x_coord', 'y_coord'])['y_coord']

# Remove column 'coordinates'
df = df.drop(['coordinates'], axis=1)
df

,type,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,type,x_coords,y_coords
0,Feature,101235305,1007,1020,1110,1401,Alpnach,OW,Point,2662391.000,1198346.000
1,Feature,101235307,1004,1020,1110,1401,Alpnach,OW,Point,2662273.495,1198664.438
2,Feature,101235308,1004,1020,1122,1401,Alpnach,OW,Point,2663069.775,1200010.861
3,Feature,101235310,1004,1020,1110,1401,Alpnach,OW,Point,2662170.316,1198226.835
4,Feature,101235311,1004,1020,1110,1401,Alpnach,OW,Point,2662135.736,1198271.709
...,...,...,...,...,...,...,...,...,...,...,...
21923,Feature,9082198,1004,1040,0,1407,Sarnen,OW,Point,2661576.416,1194345.086
21924,Feature,9082200,1004,1060,0,1407,Sarnen,OW,Point,2661535.114,1194255.211
21925,Feature,9082201,1004,1060,0,1407,Sarnen,OW,Point,2661735.402,1194681.900
21926,Feature,9082202,1004,1060,0,1407,Sarnen,OW,Point,2661570.119,1194122.619


## Create WGS84 coordinates

In [12]:
# Create geodataframe and calculate latitude and longitude
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x_coords'], df['y_coords']), crs="EPSG:2056")

# Convert the Swiss LV95 coordinates to lat & lon
gdf = gdf.to_crs(epsg=4326)

# Get Latitude and Longitude
df['latitude'] = gdf['geometry'].y
df['longitude'] = gdf['geometry'].x

# Show data
df

,type,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,type,x_coords,y_coords,latitude,longitude
0,Feature,101235305,1007,1020,1110,1401,Alpnach,OW,Point,2662391.000,1198346.000,46.933279,8.257952
1,Feature,101235307,1004,1020,1110,1401,Alpnach,OW,Point,2662273.495,1198664.438,46.936154,8.256452
2,Feature,101235308,1004,1020,1122,1401,Alpnach,OW,Point,2663069.775,1200010.861,46.948190,8.267095
3,Feature,101235310,1004,1020,1110,1401,Alpnach,OW,Point,2662170.316,1198226.835,46.932228,8.255038
4,Feature,101235311,1004,1020,1110,1401,Alpnach,OW,Point,2662135.736,1198271.709,46.932635,8.254590
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21923,Feature,9082198,1004,1040,0,1407,Sarnen,OW,Point,2661576.416,1194345.086,46.897368,8.246714
21924,Feature,9082200,1004,1060,0,1407,Sarnen,OW,Point,2661535.114,1194255.211,46.896563,8.246160
21925,Feature,9082201,1004,1060,0,1407,Sarnen,OW,Point,2661735.402,1194681.900,46.900383,8.248846
21926,Feature,9082202,1004,1060,0,1407,Sarnen,OW,Point,2661570.119,1194122.619,46.895367,8.246602


## Plot subset of buildings

In [14]:
# Subset
df_sub = df.loc[df['municipalityName'] == 'Kerns'].sample(100).dropna()
df_sub

# Create the map
m = folium.Map(location=[df_sub['latitude'].mean(), df_sub['longitude'].mean()], zoom_start=15)

# Add points to the map
for idx, row in df_sub.iterrows():
    folium.Marker(location=([row['latitude'], 
                            row['longitude']]),
                  popup=row['buildingCategory']).add_to(m)

# Display the map
m

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [7]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.2.0-1019-azure
Datetime: 2024-03-12 20:32:21
Python Version: 3.10.13
-----------------------------------
